In [1]:
import sys
import torch
import torch.nn
import jax
import jax.numpy as jnp
import numpy as np

from clu import parameter_overview

2022-11-30 16:15:25.164434: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-11-30 16:15:25.164477: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2022-11-30 16:15:25.164481: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


# Modules needed to be verified


## Basic Building Blocks (bb)
all the components in the layers file

- FlaxBertForSequenceClassification
    - FlaxBertForSequenceClassificationModule
        - FlaxBertModule
            - FlaxBertEmbeddings
                - Add
                - LayerNorm
                - Dropout
            - FlaxBertEncoder
                - FlaxBertLayerCollection
                    - FlaxBertLayer
                        - FlaxBertAttention
                            - FlaxBertSelfAttention(bb only)
                            - FlaxBertSelfOutput(bb only)
                        - FlaxBertIntermediate(bb only)
                        - FlaxBertOutput(bb only)
                    - FlaxBertCheckpointLayer(cond.)
            - FlaxBertPooler(bb only)
        - Dropout
        - Dense

## Basic Building Block verification.
- FlaxBertSelfAttention vs.     BertSelfAttention
- FlaxBertSelfOutput    vs.     BertSelfOutput
- FlaxBertIntermediate  vs.     BertIntermediate
- FlaxBertOutput        vs.     BertOutput
- FlaxBertPooler        vs.     BertPooler

In [2]:
from bert_torch.BERT import *
from bert.modeling_flax_bert import *
import bert.modeling_flax_bert as layers
from transformers import BertConfig, BertTokenizer

In [3]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [4]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
configuration = BertConfig()

In [5]:
text_batch = ["Hello world!"]
inputs = tokenizer(text_batch)
input_ids = jnp.array(inputs["input_ids"])
token_ids = jnp.array(inputs["token_type_ids"])
attention_mask = jnp.array(inputs["attention_mask"])
position_ids = jnp.arange(input_ids.shape[0])

pt_inputs = tokenizer(text_batch, return_tensors='pt')
pt_input_ids = pt_inputs["input_ids"]
pt_token_ids = pt_inputs["token_type_ids"]
pt_attention_mask = pt_inputs["attention_mask"]
pt_position_ids = torch.tensor(np.arange(pt_input_ids.shape[0]))

print(input_ids, pt_input_ids)
print(position_ids, pt_position_ids)

[[ 101 7592 2088  999  102]] tensor([[ 101, 7592, 2088,  999,  102]])
[0] tensor([0])


In [6]:
pt_layer = BertEmbeddings(configuration).eval()
f_layer = FlaxBertEmbeddings(configuration)

In [7]:
for k,v in pt_layer.named_parameters():
    print(k, v.shape)

vf = {'params':
          {'word_embeddings':
               {'embedding': None},
           'position_embeddings':
               {'embedding': None},
           'token_type_embeddings':
               {'embedding': None},
           'LayerNorm':
               {'bias': None,
                'scale': None},
        }
      }
for k,v in pt_layer.named_parameters():
    if k == 'word_embeddings.weight':
        vf['params']['word_embeddings']['embedding'] = v.detach().numpy()
    if k == 'position_embeddings.weight':
        vf['params']['position_embeddings']['embedding'] = v.detach().numpy()
    if k == 'token_type_embeddings.weight':
        vf['params']['token_type_embeddings']['embedding'] = v.detach().numpy()
    if k == 'LayerNorm.weight':
        vf['params']['LayerNorm']['scale'] = v.detach().numpy()
    if k == 'LayerNorm.bias':
        vf['params']['LayerNorm']['bias'] = v.detach().numpy()

print(vf)


word_embeddings.weight torch.Size([30522, 768])
position_embeddings.weight torch.Size([512, 768])
token_type_embeddings.weight torch.Size([2, 768])
LayerNorm.weight torch.Size([768])
LayerNorm.bias torch.Size([768])
{'params': {'word_embeddings': {'embedding': array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.02733476, -0.13386883, -1.1515571 , ..., -0.974222  ,
         0.32135013, -1.0175889 ],
       [-1.0327494 , -0.3484202 ,  1.3928472 , ...,  0.6928705 ,
        -0.36489737,  0.07088251],
       ...,
       [ 0.5125529 , -0.18763718,  1.4236608 , ..., -1.0472057 ,
        -0.1026423 ,  0.14267415],
       [ 1.6064404 , -0.9257208 , -1.04948   , ...,  0.23986971,
        -0.22128616, -1.3424199 ],
       [ 0.9218716 , -0.1787405 ,  0.2866925 , ..., -0.18176642,
        -1.4434109 ,  0.78884804]], dtype=float32)}, 'position_embeddings': {'embedding': array([[-0.31082064,  1.4829938 ,  1.1821117 , ..., -1.7082491 ,
       

In [18]:
# vf = f_layer.init(jax.random.PRNGKey(0), input_ids, token_ids,  position_ids, attention_mask)
f_out = f_layer.apply(vf, input_ids, token_ids, position_ids, attention_mask)
pt_out = pt_layer(input_ids=pt_input_ids, token_type_ids=pt_token_ids, position_ids=pt_position_ids)

In [19]:
print(vf['params']['word_embeddings']['embedding'].shape)
print(f_out.shape)
print(pt_out.shape)
print(np.abs(pt_out.detach().numpy() - f_out).sum())

(30522, 768)
(1, 5, 768)
torch.Size([1, 5, 768])
4319.8057


In [20]:
cam = np.random.rand(*f_out.shape)
f_cam = jnp.array( cam / cam.sum())
pt_cam = torch.tensor(cam / cam.sum())

f_cam1, f_cam2 = f_layer.apply(vf, f_cam, input_ids, token_ids, position_ids, attention_mask, method=f_layer.relprop)
kwargs = {'alpha': 1}
pt_cam1, pt_cam2 = pt_layer.relprop(pt_cam, **kwargs)


print(np.abs(np.array(f_cam1) - pt_cam1.detach().numpy()).sum())
print(np.abs(np.array(f_cam2) - pt_cam2.detach().numpy()).sum())
print(np.abs(np.array(f_cam1) - pt_cam2.detach().numpy()).sum())
print(np.sum(f_cam1) + np.sum(f_cam2))
print(np.sum(pt_cam1.detach().numpy()) + np.sum(pt_cam2.detach().numpy()))
print(f_cam2.shape)

relprop 2
2.5885494
2.6496449
2.849807
1.0
1.0
(1, 5, 768)


In [208]:
print(parameter_overview.get_parameter_overview(vf))
print(pt_layer, count_parameters(pt_layer))

+----------------------------------------+--------------+------------+----------+-------+
| Name                                   | Shape        | Size       | Mean     | Std   |
+----------------------------------------+--------------+------------+----------+-------+
| params/LayerNorm/bias                  | (768,)       | 768        | 0.0      | 0.0   |
| params/LayerNorm/scale                 | (768,)       | 768        | 1.0      | 0.0   |
| params/position_embeddings/embedding   | (512, 768)   | 393,216    | -0.00125 | 0.999 |
| params/token_type_embeddings/embedding | (2, 768)     | 1,536      | -0.0142  | 1.01  |
| params/word_embeddings/embedding       | (30522, 768) | 23,440,896 | 0.000289 | 1.0   |
+----------------------------------------+--------------+------------+----------+-------+
Total: 23,837,184
BertEmbeddings(
  (word_embeddings): Embedding(30522, 768, padding_idx=0)
  (position_embeddings): Embedding(512, 768)
  (token_type_embeddings): Embedding(2, 768)
  (Layer

In [158]:
pt_layer = BertEncoder(configuration).eval()
f_layer = FlaxBertEncoder(configuration)

In [159]:
for k,v in pt_layer.named_parameters():
    print(k, v.shape)

layer.0.attention.self.query.weight torch.Size([768, 768])
layer.0.attention.self.query.bias torch.Size([768])
layer.0.attention.self.key.weight torch.Size([768, 768])
layer.0.attention.self.key.bias torch.Size([768])
layer.0.attention.self.value.weight torch.Size([768, 768])
layer.0.attention.self.value.bias torch.Size([768])
layer.0.attention.output.dense.weight torch.Size([768, 768])
layer.0.attention.output.dense.bias torch.Size([768])
layer.0.attention.output.LayerNorm.weight torch.Size([768])
layer.0.attention.output.LayerNorm.bias torch.Size([768])
layer.0.intermediate.dense.weight torch.Size([3072, 768])
layer.0.intermediate.dense.bias torch.Size([3072])
layer.0.output.dense.weight torch.Size([768, 3072])
layer.0.output.dense.bias torch.Size([768])
layer.0.output.LayerNorm.weight torch.Size([768])
layer.0.output.LayerNorm.bias torch.Size([768])
layer.1.attention.self.query.weight torch.Size([768, 768])
layer.1.attention.self.query.bias torch.Size([768])
layer.1.attention.self.k